In [1]:
import torch
import pandas as pd
import torch.nn as nn 
import torch.optim as opt
from sklearn.preprocessing import LabelEncoder , StandardScaler , OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("Dataset/Titanic/train.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
509,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,NaN,S
336,337,0,1,"Pears, Mr. Thomas Clinton",male,29.0,1,0,113776,66.6000,C2,S
79,80,1,3,"Dowdell, Miss. Elizabeth",female,30.0,0,0,364516,12.4750,NaN,S
841,842,0,2,"Mudd, Mr. Thomas Charles",male,16.0,0,0,S.O./P.P. 3,10.5000,NaN,S
123,124,1,2,"Webber, Miss. Susan",female,32.5,0,0,27267,13.0000,E101,S


### Preprocess the data

In [3]:
# Preprocss the datset
df = df.drop(columns= ['Name' , 'Cabin' ,"PassengerId" , "Ticket"])

# Split the dataset
y = df['Survived']
X = df.drop(columns= ['Survived'])
X_train , x_test , y_train , y_test = train_test_split(X, y , test_size= 0.2 , random_state= 42)
X_train.shape , x_test.shape

# impuation for Age
Si = SimpleImputer(strategy= "mean")
X_train['Age'] = Si.fit_transform(X_train[['Age']])
x_test['Age'] = Si.transform(x_test[["Age"]])

# Impuation for Embarked
X_train = X_train.fillna({'Embarked': 'missing'})  # Replace with 'missing' instead of dropping
x_test = x_test.fillna({'Embarked': 'missing'})

# OHE
object_columns = X_train.select_dtypes(include=['object']).columns.tolist()
ohe = OneHotEncoder(sparse_output= False)

X_train_ohe_encoder = ohe.fit_transform(X_train[object_columns])
x_test_ohe_encoder = ohe.transform(x_test[object_columns])

X_train_ohe_df = pd.DataFrame(X_train_ohe_encoder , columns= ohe.get_feature_names_out(object_columns), index= X_train.index)
x_test_ohe_df = pd.DataFrame(x_test_ohe_encoder , columns= ohe.get_feature_names_out(object_columns) , index= x_test.index)

X_train = pd.concat([X_train , X_train_ohe_df] , axis= 1)
x_test = pd.concat([x_test , x_test_ohe_df] , axis= 1)

X_train = X_train.drop(object_columns , axis= 1)
x_test = x_test.drop(object_columns , axis= 1)

# Scale Values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)



In [4]:
# Label Encoder
LE = LabelEncoder()
y_train = LE.fit_transform(y_train)
y_test = LE.transform(y_test)
X_train.shape , y_train.shape

((712, 11), (712,))

### Started with NN Module

In [5]:
# Converting into tensor
import numpy as np 
X_train = np.array(X_train)
x_test = np.array(x_test)
X_train = torch.from_numpy(X_train)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)
y_train = torch.from_numpy(y_train)

In [6]:
# Model Creation
class NeuralNetwork(nn.Module):
    def __init__(self , Feature_count):
        super().__init__() 
        self.linear1 = nn.Linear(Feature_count , 5)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear( 5 ,3 )
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(3 ,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self , features):
        out = self.linear1(features)
        out = self.relu1(out)
        out = self.linear2(out)
        out = self.relu2(out)
        out = self.linear3(out)
        out = self.sigmoid(out)
        
        return out
        


In [7]:
X_train = X_train.to(dtype= torch.float32)
x_test = x_test.to(dtype= torch.float32)

# Creating model instance
model = NeuralNetwork(X_train.shape[1])
output = model(X_train) 

In [8]:
# using Sequentailly container 
# Model Creation
class NeuralNetworkFinal(nn.Module):
    def __init__(self , Feature_count):
        super().__init__() 
        self.network = nn.Sequential(       
        nn.Linear(Feature_count , 5),
        nn.ReLU(),
        nn.Linear( 5 ,3 ),
        nn.ReLU(),
        nn.Linear(3 ,1),
        nn.Sigmoid()
        )

        
    def forward(self , features):
        out = self.network(features)   
        return out

loss_funtion = nn.BCELoss()
    

In [9]:
from torchinfo import summary
summary(model , (712, 11))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [712, 1]                  --
├─Linear: 1-1                            [712, 5]                  60
├─ReLU: 1-2                              [712, 5]                  --
├─Linear: 1-3                            [712, 3]                  18
├─ReLU: 1-4                              [712, 3]                  --
├─Linear: 1-5                            [712, 1]                  4
├─Sigmoid: 1-6                           [712, 1]                  --
Total params: 82
Trainable params: 82
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.06
Input size (MB): 0.03
Forward/backward pass size (MB): 0.05
Params size (MB): 0.00
Estimated Total Size (MB): 0.08

In [10]:
lr = 0.1
epoch = 20
X_train = X_train.to(dtype= torch.float32)
x_test = x_test.to(dtype= torch.float32)
y_test = y_test.to(dtype= torch.float32)
y_train = y_train.to(dtype= torch.float32)

# Creating model instance
model = NeuralNetworkFinal(X_train.shape[1])
optimiser = opt.Adam(model.parameters() , lr)
 

for ep in range(epoch):
    # Forward pass
    y_pred = model(X_train)
    
    # Loss calculation
    loss = loss_funtion(y_pred , y_train.view(-1 ,1 ))
    
    # clear gradient 
    optimiser.zero_grad()
    
    # Backprog loss
    loss.backward()
    
    # paramter upadates
    optimiser.step()
    
    print(f"Epoch : {ep+1} , loss : {loss}")
    

Epoch : 1 , loss : 0.6549752950668335
Epoch : 2 , loss : 0.6182204484939575
Epoch : 3 , loss : 0.5677930116653442
Epoch : 4 , loss : 0.5283716320991516
Epoch : 5 , loss : 0.5044397711753845
Epoch : 6 , loss : 0.481413334608078
Epoch : 7 , loss : 0.47606542706489563
Epoch : 8 , loss : 0.4680674374103546
Epoch : 9 , loss : 0.4529365301132202
Epoch : 10 , loss : 0.4485081732273102
Epoch : 11 , loss : 0.4529380798339844
Epoch : 12 , loss : 0.4483601748943329
Epoch : 13 , loss : 0.4417493939399719
Epoch : 14 , loss : 0.4411925971508026
Epoch : 15 , loss : 0.4401077926158905
Epoch : 16 , loss : 0.4354932904243469
Epoch : 17 , loss : 0.43287190794944763
Epoch : 18 , loss : 0.43011072278022766
Epoch : 19 , loss : 0.42615342140197754
Epoch : 20 , loss : 0.42356324195861816


In [11]:
# Evaluate the model

with torch.no_grad():
    y_pred = model.forward(x_test)
    y_pred = (y_pred > 0.6).float()  # threshold = 0.6
    
    acc = (y_pred == y_test).float().mean() # to calculate accuracy
    
    print(f"Accuracy: {acc}")
    

Accuracy: 0.53531414270401
